In [ ]:
!pip install finance-datareader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from datetime import datetime, timedelta
import random

day_num = 32

In [ ]:
from keras.models import load_model

model6 = load_model('drive/MyDrive/Colab Notebooks/change4.h5')
model7 = load_model('drive/MyDrive/Colab Notebooks/change5.h5')
model8 = load_model('drive/MyDrive/Colab Notebooks/change6.h5')
model9 = load_model('drive/MyDrive/Colab Notebooks/pi90_daq500_label10_epoch40.h5')
model10 = load_model('drive/MyDrive/Colab Notebooks/pi90_daq500_label10_epoch50.h5')

In [ ]:
def convert_image(data, num_images, day_num, name, code, save = False, test = False):

    def alpha_blend(color1, color2, alpha):
        return [
            int((color1[0] * alpha + color2[0] * (1 - alpha))),
            int((color1[1] * alpha + color2[1] * (1 - alpha))),
            int((color1[2] * alpha + color2[2] * (1 - alpha))),
            255,
        ]


    # 해당 종목의 주가 데이터 - 100이 요구되는 이미지 수보다 적으면(상장직후는 변동이 심할 수있으니 제외)
    if num_images > len(data) - 60 - day_num:
        num_images = len(data) - 60 - day_num

    #return_datas는 이미지, 라벨링(상승이면1 하락이면0), 변동률, 날짜를 담고있음
    return_datas = []

    image_idx = 1
    while(image_idx <= num_images):

        if image_idx != 1:
            stock_data = data[-image_idx - day_num + 1 : -image_idx + 1]
        else:
            stock_data = data[-day_num:]
        # 이미지 크기 설정
        width, height = day_num * 3, 96

        # High, Low, Close, Volume 데이터 정규화
        high_prices = stock_data['High'].values
        low_prices = stock_data['Low'].values
        close_prices = stock_data['Close'].values
        volume_data = stock_data['Volume'].values

        high_prices_norm = (high_prices - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
        low_prices_norm = (low_prices - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
        close_prices_norm = (close_prices - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
        volume_data_norm = height * (volume_data) / (2 * (np.max(volume_data)))

        if len(data) <= abs(-image_idx - day_num - 19):
            break

        # 이동평균선 추가를 위한 stock_data 생성
        if image_idx != 1:
            stock_data_ma20 = data[-image_idx - day_num - 19: -image_idx + 1]
        else:
            stock_data_ma20 = data[-image_idx - day_num - 19: ]

        stock_data_ma20 = stock_data_ma20.copy()
        stock_data_ma20['MA20'] = stock_data_ma20['Close'].rolling(window=20).mean()
        stock_data_ma20.dropna(inplace=True)

        # 이동평균선 값을 정규화
        stock_data_ma20['MA20_norm'] = (
                (stock_data_ma20['MA20'] - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
            )

        if len(data) <= abs(-image_idx - day_num - 39):
            break

        if image_idx != 1:
            stock_data_ma40 = data[-image_idx - day_num - 39: -image_idx + 1]
        else:
            stock_data_ma40 = data[-image_idx - day_num - 39: ]

        stock_data_ma40 = stock_data_ma40.copy()
        stock_data_ma40['MA40'] = stock_data_ma40['Close'].rolling(window=40).mean()
        stock_data_ma40.dropna(inplace=True)

        # 이동평균선 값을 정규화
        stock_data_ma40['MA40_norm'] = (
                (stock_data_ma40['MA40'] - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
            )

        if len(data) <= abs(-image_idx - day_num - 59):
            break

        if image_idx != 1:
            stock_data_ma60 = data[-image_idx - day_num - 59: -image_idx + 1]
        else:
            stock_data_ma60 = data[-image_idx - day_num - 59: ]

        stock_data_ma60 = stock_data_ma60.copy()
        stock_data_ma60['MA60'] = stock_data_ma60['Close'].rolling(window=60).mean()
        stock_data_ma60.dropna(inplace=True)

        # 이동평균선 값을 정규화
        stock_data_ma60['MA60_norm'] = (
                (stock_data_ma60['MA60'] - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
            )

        # 96x96 이미지 생성
        img = np.zeros((height, width, 4), dtype=np.uint8)  # 4 channels for RGBA

        fail_generate = False
        for day in range(day_num):
            high_price = int(high_prices_norm[day] * (height - 1))
            low_price = int(low_prices_norm[day] * (height - 1))
            close_price = int(close_prices_norm[day] * (height - 1))
            open_price = int(
                (
                    (stock_data['Open'].values[day] - np.min(low_prices))
                    / (np.max(high_prices) - np.min(low_prices))
                )
                * (height - 1)
            )
            volume = int(volume_data_norm[day])

            x_start = day * 3

            # 시가, 종가, 고가, 저가 막대그래프
            # 막대 왼쪽, 오른쪽 그리기 (시가, 종가 범위)
            for x in range(x_start, x_start + 3):
                if close_prices[day] > stock_data['Open'].values[day]:
                    color = [255, 0, 0, 255]  # 상승: 빨강
                    tail_color = [200, 0, 0, 255]  # 상승 시 꼬리: 어두운 빨강
                elif close_prices[day] < stock_data['Open'].values[day]:
                    color = [0, 0, 255, 255]  # 하락: 파랑
                    tail_color = [0, 0, 200, 255]  # 하락 시 꼬리: 어두운 파랑
                else: # 시가와 종가가 같을경우 어제보다 같거나 올랐으면 빨강, 아니면 파랑
                    if 0 <= stock_data['Change'].values[day]:
                        color = [255, 0, 0, 255]  # 상승: 빨강
                        tail_color = [200, 0, 0, 255]  # 상승 시 꼬리: 어두운 빨강
                    else:
                        color = [0, 0, 255, 255]  # 하락: 파랑
                        tail_color = [0, 0, 200, 255]  # 하락 시 꼬리: 어두운 파랑

                for y in range(min(open_price, close_price), max(open_price, close_price) + 1):
                    img[height - 1 - y, x] = color

            # 꼬리 그리기 (High, Low 범위)
            for x in range(x_start + 1, x_start + 2):
                for y in range(low_price, high_price + 1):
                    if img[height - 1 - y, x][0] != 255 and img[height - 1 - y, x][2] != 255:
                        img[height - 1 - y, x] = tail_color

            # 거래량 그리기
            for x in range(x_start, x_start + 3):
                overlapped = height - volume
                if overlapped >= 0 and overlapped < height:
                    img[overlapped:, x] = [
                        alpha_blend(img[y, x], [128, 128, 128, 255], 0.45)
                        for y in range(overlapped, height)
                    ]

            # 20일 이동평균선 그리기
            ma20_norm = stock_data_ma20.iloc[day]['MA20_norm']

            for x in range(x_start, x_start + 3):
                if not np.isnan(ma20_norm):
                    ma20_y = int(ma20_norm * (height - 1))
                    if 0 <= ma20_y < height:
                        for y in range(ma20_y - 1, ma20_y + 2):  # 이동평균선 주변에 블렌딩 처리
                            if 0 <= y < height:
                                new_color = alpha_blend([255, 255, 0, 255], img[height - 1 - y, x], 0.6)  # 노란색으로 변경
                                img[height - 1 - y, x] = new_color

            # 40일 이동평균선 그리기
            ma40_norm = stock_data_ma40.iloc[day]['MA40_norm']

            for x in range(x_start, x_start + 3):
                if not np.isnan(ma40_norm):
                    ma40_y = int(ma40_norm * (height - 1))
                    if 0 <= ma40_y < height:
                        for y in range(ma40_y - 1, ma40_y + 2):  # 이동평균선 주변에 블렌딩 처리
                            if 0 <= y < height:
                                new_color = alpha_blend([128, 0, 128, 255], img[height - 1 - y, x], 0.6)  # 보라색으로 변경
                                img[height - 1 - y, x] = new_color


            # 60일 이동평균선 그리기
            ma60_norm = stock_data_ma60.iloc[day]['MA60_norm']

            for x in range(x_start, x_start + 3):
                if not np.isnan(ma60_norm):
                    ma60_y = int(ma60_norm * (height - 1))
                    if 0 <= ma60_y < height:
                        for y in range(ma60_y - 1, ma60_y + 2):  # 이동평균선 주변에 블렌딩 처리
                            if 0 <= y < height:
                                new_color = alpha_blend([0, 255, 0, 255], img[height - 1 - y, x], 0.6)  # 초록색으로 변경
                                img[height - 1 - y, x] = new_color

        #이미지가 day_num(그려지기 원하는 거래일 수)미만으로 그려졌으면
        if fail_generate:
            image_idx += 1
            continue

        if save :
            # 이미지 저장
            img = Image.fromarray(img, 'RGBA')
            img.save(f'stock_data_image_{image_idx}.png')


        return_datas.append((img, name, code, stock_data.index[-1]))

        image_idx += 1


    return return_datas

In [ ]:
def preprocess_data(data):
    # 0이나 NaN을 포함하는 행 제거
    data = data[data['Volume'] != 0]
    data = data.dropna(how='any')

    return data

In [ ]:
df = pd.read_excel('drive/MyDrive/Colab Notebooks/kospi.xlsx', usecols=[0, 1], dtype={1: str})
# df = df.sample(n=400, random_state=42)
# 첫 번째 열과 두 번째 열을 Nx2 리스트로 변환
stock_codes = df.values.tolist()

# 결과 출력
print(stock_codes)

[['095570', 'AJ네트웍스'], ['006840', 'AK홀딩스'], ['027410', 'BGF'], ['282330', 'BGF리테일'], ['138930', 'BNK금융지주'], ['001460', 'BYC'], ['001465', 'BYC우'], ['001040', 'CJ'], ['079160', 'CJ CGV'], ['00104K', 'CJ4우(전환)'], ['000120', 'CJ대한통운'], ['011150', 'CJ씨푸드'], ['011155', 'CJ씨푸드1우'], ['001045', 'CJ우'], ['097950', 'CJ제일제당'], ['097955', 'CJ제일제당 우'], ['000480', 'CR홀딩스'], ['000590', 'CS홀딩스'], ['012030', 'DB'], ['016610', 'DB금융투자'], ['005830', 'DB손해보험'], ['000990', 'DB하이텍'], ['139130', 'DGB금융지주'], ['001530', 'DI동일'], ['000210', 'DL'], ['001880', 'DL건설'], ['000215', 'DL우'], ['375500', 'DL이앤씨'], ['37550L', 'DL이앤씨2우(전환)'], ['37550K', 'DL이앤씨우'], ['007340', 'DN오토모티브'], ['004840', 'DRB동일'], ['155660', 'DSR'], ['069730', 'DSR제강'], ['017940', 'E1'], ['365550', 'ESR켄달스퀘어리츠'], ['383220', 'F&F'], ['007700', 'F&F홀딩스'], ['114090', 'GKL'], ['078930', 'GS'], ['006360', 'GS건설'], ['001250', 'GS글로벌'], ['007070', 'GS리테일'], ['078935', 'GS우'], ['012630', 'HDC'], ['039570', 'HDC랩스'], ['089470', 'HDC현대EP'], ['294870', 'H

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
up_images = []
up_images_info = []

def process():
    global up_images, up_images_info

    for n in range(0, len(stock_codes), 50):
        result_images = []
        for i, ticker in enumerate(stock_codes[n:min(n+50, len(stock_codes))]):
            code, name = ticker[0], ticker[1]
            print(f"{n+i+1}번째 종목 {name} 추출중...   ")

            try:
                data = preprocess_data(fdr.DataReader(code, '2023-03-01'))  # 각 종목의 데이터 가져오기
                images = convert_image(data, 1, day_num, name, code)
                result_images += images
                print(f"{len(images)}개 추출완료. 누적{len(result_images)}개")
            except IndexError:
                print(f"{name}에서 IndexError가 발생했습니다. 다음 종목으로 이동합니다.")
                continue

        if len(result_images) == 0:
            continue
        test_images = [image[0][:, :, :3] for image in result_images]
        test_images = np.array(test_images).reshape(-1, 96, 96, 3) / 255.0

        pred_probs6 = model6.predict(test_images)
        pred_probs7 = model7.predict(test_images)
        pred_probs8 = model8.predict(test_images)
        pred_probs9 = model9.predict(test_images)
        pred_probs10 = model10.predict(test_images)

        # 두 모델 같은 라벨이 하락으로 정답이라고 했을때

        selected_images = []
        selected_images_info = []

        for i in range(len(test_images)):
            pred_probs6_max_idx = np.argmax(pred_probs6[i])
            pred_probs7_max_idx = np.argmax(pred_probs7[i])
            pred_probs8_max_idx = np.argmax(pred_probs8[i])
            pred_probs9_max_idx = np.argmax(pred_probs9[i])
            pred_probs10_max_idx = np.argmax(pred_probs10[i])

            if pred_probs6_max_idx == pred_probs7_max_idx == pred_probs8_max_idx == pred_probs9_max_idx == pred_probs10_max_idx and 0.5 < pred_probs6[i][1] and 0.5 <= pred_probs6[i][pred_probs6_max_idx] <= 0.98 and 0.5 <= pred_probs7[i][pred_probs7_max_idx] <= 0.98 and 0.75 <= pred_probs8[i][pred_probs8_max_idx] <= 1 and 0.54 <= pred_probs9[i][pred_probs9_max_idx] <= 0.97 and 0.75 <= pred_probs10[i][pred_probs10_max_idx] <= 1:
                selected_images.append(test_images[i])
                selected_images_info.append(result_images[i][1:])

        selected_images = np.array(selected_images)

        print(len(selected_images))
        for i in selected_images_info:
            print(i[0], str(i[-1])[:10])

        up_images.append(selected_images)
        up_images_info.append(selected_images_info)

In [ ]:
process()

1번째 종목 AJ네트웍스 추출중...   
1개 추출완료. 누적1개
2번째 종목 AK홀딩스 추출중...   
1개 추출완료. 누적2개
3번째 종목 BGF 추출중...   
1개 추출완료. 누적3개
4번째 종목 BGF리테일 추출중...   
1개 추출완료. 누적4개
5번째 종목 BNK금융지주 추출중...   
1개 추출완료. 누적5개
6번째 종목 BYC 추출중...   
1개 추출완료. 누적6개
7번째 종목 BYC우 추출중...   
1개 추출완료. 누적7개
8번째 종목 CJ 추출중...   
1개 추출완료. 누적8개
9번째 종목 CJ CGV 추출중...   
1개 추출완료. 누적9개
10번째 종목 CJ4우(전환) 추출중...   
1개 추출완료. 누적10개
11번째 종목 CJ대한통운 추출중...   
1개 추출완료. 누적11개
12번째 종목 CJ씨푸드 추출중...   
1개 추출완료. 누적12개
13번째 종목 CJ씨푸드1우 추출중...   
1개 추출완료. 누적13개
14번째 종목 CJ우 추출중...   
1개 추출완료. 누적14개
15번째 종목 CJ제일제당 추출중...   
1개 추출완료. 누적15개
16번째 종목 CJ제일제당 우 추출중...   
1개 추출완료. 누적16개
17번째 종목 CR홀딩스 추출중...   
1개 추출완료. 누적17개
18번째 종목 CS홀딩스 추출중...   
1개 추출완료. 누적18개
19번째 종목 DB 추출중...   
1개 추출완료. 누적19개
20번째 종목 DB금융투자 추출중...   
1개 추출완료. 누적20개
21번째 종목 DB손해보험 추출중...   
1개 추출완료. 누적21개
22번째 종목 DB하이텍 추출중...   
1개 추출완료. 누적22개
23번째 종목 DGB금융지주 추출중...   
1개 추출완료. 누적23개
24번째 종목 DI동일 추출중...   
1개 추출완료. 누적24개
25번째 종목 DL 추출중...   
1개 추출완료. 누적25개
26번째 종목 DL건설 추출중...   
1개 추출완료. 누적26개
27번

2/2 [==============================] - 0s 17ms/step
0
51번째 종목 HD현대건설기계 추출중...   
1개 추출완료. 누적1개
52번째 종목 HD현대에너지솔루션 추출중...   
1개 추출완료. 누적2개
53번째 종목 HD현대인프라코어 추출중...   
1개 추출완료. 누적3개
54번째 종목 HD현대일렉트릭 추출중...   
1개 추출완료. 누적4개
55번째 종목 HD현대중공업 추출중...   
1개 추출완료. 누적5개
56번째 종목 HJ중공업 추출중...   
1개 추출완료. 누적6개
57번째 종목 HL D&I 추출중...   
1개 추출완료. 누적7개
58번째 종목 HLB글로벌 추출중...   
1개 추출완료. 누적8개
59번째 종목 HL만도 추출중...   
1개 추출완료. 누적9개
60번째 종목 HL홀딩스 추출중...   
1개 추출완료. 누적10개
61번째 종목 HMM 추출중...   
1개 추출완료. 누적11개
62번째 종목 HSD엔진 추출중...   
1개 추출완료. 누적12개
63번째 종목 HS애드 추출중...   
1개 추출완료. 누적13개
64번째 종목 IHQ 추출중...   
0개 추출완료. 누적13개
65번째 종목 JB금융지주 추출중...   
1개 추출완료. 누적14개
66번째 종목 JW생명과학 추출중...   
1개 추출완료. 누적15개
67번째 종목 JW중외제약 추출중...   
1개 추출완료. 누적16개
68번째 종목 JW중외제약2우B 추출중...   
1개 추출완료. 누적17개
69번째 종목 JW중외제약우 추출중...   
1개 추출완료. 누적18개
70번째 종목 JW홀딩스 추출중...   
1개 추출완료. 누적19개
71번째 종목 KB금융 추출중...   
1개 추출완료. 누적20개
72번째 종목 KB스타리츠 추출중...   
1개 추출완료. 누적21개
73번째 종목 KCC 추출중...   
1개 추출완료. 누적22개
74번째 종목 KCC글라스 추출중...   
1개 추출완료. 누적23

In [ ]:
for up in up_images_info:
    for i in up:
        print(i[0], str(i[2])[:10])

진흥기업우B 2023-09-13
